In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
from kaggle_secrets import UserSecretsClient

# W&B login via Kaggle secrets
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb-key"))

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(DD)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CentralityEncoder(nn.Module):
    def __init__(self, in_dim=1, out_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, out_dim)
        )
    def forward(self, c):
        return self.net(c)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = CentralityEncoder(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent, edge_index, batch):
        x = torch.cat([x, self.cent_enc(cent)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def _minmax_norm(t):
    if len(t) == 0:
        return t
    vmin, vmax = t.min(), t.max()
    if (vmax - vmin) > 0:
        return (t - vmin) / (vmax - vmin)
    return torch.zeros_like(t)

def make_centrality_transform():
    def transform(data):
        if getattr(data, "x", None) is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        G = to_networkx(data, to_undirected=True)
        cent = torch.zeros((data.num_nodes, 1), dtype=torch.float)
        if G.number_of_nodes() > 0:
            try:
                ev = nx.eigenvector_centrality_numpy(G)
            except Exception:
                ev = {n: 0.0 for n in G.nodes}
            ev_tensor = torch.tensor([ev.get(n, 0.0) for n in range(data.num_nodes)], dtype=torch.float)
            cent[:, 0] = _minmax_norm(ev_tensor)
        data.cent = cent
        return data
    return transform

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root="data/TUDataset", name="DD", transform=make_centrality_transform())
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GCN(dataset.num_features if dataset.num_features else 1, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"DD_GCN_Centrality_PROTEINS_seed_{seed}",
        group="GCN_PROTEINS_Centrality",
        job_type="centrality_only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Centrality_eigenvector",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss, correct, count = 0.0, 0, 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.cent, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return total_loss / len(train_loader), correct / count

    def test(loader):
        model.eval()
        correct, count = 0, 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.cent, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return correct / count

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({
            "epoch": epoch,
            "loss": loss,
            "train_acc": train_acc,
            "test_acc": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galsalman (noynetanel1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_175546-2diuekik
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run DD_GCN_Centrality_PROTEINS_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal
wandb: 🚀 View run at https://wandb.a

Seed 1 | Epoch 001 | Loss: 0.6845 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 002 | Loss: 0.6798 | Train Acc: 0.5857 | Test Acc: 0.5874
Seed 1 | Epoch 003 | Loss: 0.6731 | Train Acc: 0.5942 | Test Acc: 0.5874
Seed 1 | Epoch 004 | Loss: 0.6621 | Train Acc: 0.6112 | Test Acc: 0.6027
Seed 1 | Epoch 005 | Loss: 0.6350 | Train Acc: 0.6401 | Test Acc: 0.6944
Seed 1 | Epoch 006 | Loss: 0.5998 | Train Acc: 0.7165 | Test Acc: 0.7063
Seed 1 | Epoch 007 | Loss: 0.5824 | Train Acc: 0.7148 | Test Acc: 0.7046
Seed 1 | Epoch 008 | Loss: 0.5943 | Train Acc: 0.6910 | Test Acc: 0.7063
Seed 1 | Epoch 009 | Loss: 0.5771 | Train Acc: 0.7267 | Test Acc: 0.6978
Seed 1 | Epoch 010 | Loss: 0.5646 | Train Acc: 0.7267 | Test Acc: 0.6859
Seed 1 | Epoch 011 | Loss: 0.5685 | Train Acc: 0.7182 | Test Acc: 0.7029
Seed 1 | Epoch 012 | Loss: 0.5761 | Train Acc: 0.7199 | Test Acc: 0.7267
Seed 1 | Epoch 013 | Loss: 0.5774 | Train Acc: 0.7131 | Test Acc: 0.7046
Seed 1 | Epoch 014 | Loss: 0.5656 | Train Acc: 0.72

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▆▄▄▄▃▃▄▃▃▃▃▃▃▂▃▂▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▂▂▁
wandb:  test_acc ▁▁▁▅▅▅▅▅▅▆▅▅▆▅▅▆▆▆▅▆▅▆▆▅▆▆▇▆▆▇▇████▇█▇▆▇
wandb: train_acc ▁▁▁▂▆▅▆▆▆▆▆▆▆▆▆▆▆▆▇▆▇▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.51138
wandb:  test_acc 0.75382
wandb: train_acc 0.78098
wandb: 
wandb: 🚀 View run DD_GCN_Centrality_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal/runs/2diuekik
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_175546-2diuekik/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working

Seed 2 | Epoch 001 | Loss: 0.6858 | Train Acc: 0.5654 | Test Acc: 0.5942
Seed 2 | Epoch 002 | Loss: 0.6741 | Train Acc: 0.5789 | Test Acc: 0.5942
Seed 2 | Epoch 003 | Loss: 0.6625 | Train Acc: 0.6044 | Test Acc: 0.6978
Seed 2 | Epoch 004 | Loss: 0.6534 | Train Acc: 0.6333 | Test Acc: 0.7097
Seed 2 | Epoch 005 | Loss: 0.6225 | Train Acc: 0.6842 | Test Acc: 0.6452
Seed 2 | Epoch 006 | Loss: 0.6095 | Train Acc: 0.6774 | Test Acc: 0.7080
Seed 2 | Epoch 007 | Loss: 0.5869 | Train Acc: 0.6893 | Test Acc: 0.6927
Seed 2 | Epoch 008 | Loss: 0.5869 | Train Acc: 0.6842 | Test Acc: 0.7012
Seed 2 | Epoch 009 | Loss: 0.5870 | Train Acc: 0.6995 | Test Acc: 0.7063
Seed 2 | Epoch 010 | Loss: 0.5741 | Train Acc: 0.7012 | Test Acc: 0.7317
Seed 2 | Epoch 011 | Loss: 0.5644 | Train Acc: 0.7199 | Test Acc: 0.7029
Seed 2 | Epoch 012 | Loss: 0.5718 | Train Acc: 0.7097 | Test Acc: 0.7334
Seed 2 | Epoch 013 | Loss: 0.5643 | Train Acc: 0.7012 | Test Acc: 0.7114
Seed 2 | Epoch 014 | Loss: 0.5885 | Train Acc: 0.71

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ██▇▇▆▄▄▄▄▃▃▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▂▁▁▁▁▁▁
wandb:  test_acc ▁▁▅▆▃▅▆▆▇▆▆▆▆▇▇▆▇▇▇▇▇██▇█▇███▇▆▆█▇██████
wandb: train_acc ▁▁▂▃▅▅▅▆▆▆▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇█▇▇█▇██▇▇█▇▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.50316
wandb:  test_acc 0.76061
wandb: train_acc 0.76061
wandb: 
wandb: 🚀 View run DD_GCN_Centrality_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal/runs/d9papg0q
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_180753-d9papg0q/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss ███▇▇▅▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
wandb:  test_acc ▁▁▁▂▅▅▆▆▆▆▇▇▇▇▇▇██▇▆▇▇████▇▇█▇▇▇▇█▇█▇█▇▇
wandb: train_acc ▁▂▂▂▃▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██▇▇▇▇█▇███████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.45712
wandb:  test_acc 0.74024
wandb: train_acc 0.79966
wandb: 
wandb: 🚀 View run DD_GCN_Centrality_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal/runs/pe2g3ax0
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_231817-pe2g3ax0/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/workin

Seed 29 | Epoch 001 | Loss: 0.6822 | Train Acc: 0.5671 | Test Acc: 0.5789
Seed 29 | Epoch 002 | Loss: 0.6765 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 003 | Loss: 0.6733 | Train Acc: 0.5942 | Test Acc: 0.5789
Seed 29 | Epoch 004 | Loss: 0.6704 | Train Acc: 0.5942 | Test Acc: 0.5806
Seed 29 | Epoch 005 | Loss: 0.6640 | Train Acc: 0.5942 | Test Acc: 0.5840
Seed 29 | Epoch 006 | Loss: 0.6521 | Train Acc: 0.6163 | Test Acc: 0.6197
Seed 29 | Epoch 007 | Loss: 0.6359 | Train Acc: 0.6418 | Test Acc: 0.6180
Seed 29 | Epoch 008 | Loss: 0.6232 | Train Acc: 0.6621 | Test Acc: 0.7182
Seed 29 | Epoch 009 | Loss: 0.6140 | Train Acc: 0.6740 | Test Acc: 0.6910
Seed 29 | Epoch 010 | Loss: 0.6037 | Train Acc: 0.6740 | Test Acc: 0.7131
Seed 29 | Epoch 011 | Loss: 0.5971 | Train Acc: 0.6893 | Test Acc: 0.7114
Seed 29 | Epoch 012 | Loss: 0.6141 | Train Acc: 0.6910 | Test Acc: 0.6876
Seed 29 | Epoch 013 | Loss: 0.6006 | Train Acc: 0.6876 | Test Acc: 0.6876
Seed 29 | Epoch 014 | Loss: 0.6021 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██
wandb:      loss ████▇▆▆▅▅▄▅▅▄▄▄▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▃▂▂▁▁▁▁▁
wandb:  test_acc ▁▁▁▁▁▂▆▅▆▆▅▆▆▇▆▆▇▆▆▇▇▇▆▇▆▇▇▇█▇██████▇███
wandb: train_acc ▁▂▂▂▃▄▅▅▅▅▅▅▆▆▆▆▅▆▆▆▆▇▆▆▇▇▇▇▇▇███▇███▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.52016
wandb:  test_acc 0.75891
wandb: train_acc 0.76061
wandb: 
wandb: 🚀 View run DD_GCN_Centrality_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal/runs/pbflzsem
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_233005-pbflzsem/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/workin

Seed 30 | Epoch 001 | Loss: 0.6840 | Train Acc: 0.5586 | Test Acc: 0.5993
Seed 30 | Epoch 002 | Loss: 0.6826 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 003 | Loss: 0.6726 | Train Acc: 0.5739 | Test Acc: 0.5993
Seed 30 | Epoch 004 | Loss: 0.6730 | Train Acc: 0.5891 | Test Acc: 0.6316
Seed 30 | Epoch 005 | Loss: 0.6535 | Train Acc: 0.6163 | Test Acc: 0.6570
Seed 30 | Epoch 006 | Loss: 0.6319 | Train Acc: 0.6503 | Test Acc: 0.6978
Seed 30 | Epoch 007 | Loss: 0.6024 | Train Acc: 0.6944 | Test Acc: 0.6961
Seed 30 | Epoch 008 | Loss: 0.5947 | Train Acc: 0.7012 | Test Acc: 0.6995
Seed 30 | Epoch 009 | Loss: 0.5907 | Train Acc: 0.7080 | Test Acc: 0.7182
Seed 30 | Epoch 010 | Loss: 0.6034 | Train Acc: 0.7063 | Test Acc: 0.7182
Seed 30 | Epoch 011 | Loss: 0.5823 | Train Acc: 0.7216 | Test Acc: 0.7199
Seed 30 | Epoch 012 | Loss: 0.5857 | Train Acc: 0.7114 | Test Acc: 0.7250
Seed 30 | Epoch 013 | Loss: 0.5782 | Train Acc: 0.7097 | Test Acc: 0.7114
Seed 30 | Epoch 014 | Loss: 0.5813 | T

wandb: uploading history steps 48-48, summary, console lines 48-48
wandb: 
wandb: updating run metadata; uploading history steps 49-49, console lines 49-49
wandb: updating run metadata
wandb: updating run metadata; uploading summary
wandb: 
wandb: uploading data
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ████▇▅▅▄▅▄▄▄▅▄▄▃▃▃▃▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁
wandb:  test_acc ▁▁▁▂▃▅▅▆▆▆▆▅▆▆▆▆▆▇▇▇██▇▇▇▇▇█▅█████████▆▆
wandb: train_acc ▁▁▁▂▃▅▅▆▆▆▆▆▆▅▆▆▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇███▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 0.51391
wandb:  test_acc 0.72666
wandb: train_acc 0.75891
wandb: 
wandb: 🚀 View run DD_GCN_Centrality_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28DD%29_Gal/runs/foxs2qcd
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classifi